In [1]:
from sklearn.datasets import load_iris
import numpy as np
import umap
iris = load_iris()
umapper0 = umap.UMAP(
    n_neighbors=50, learning_rate=0.5, random_state=12345, min_dist=0.001,
    init="random", n_epochs=1,
)
print("Generating an initial embedding...")
emb0 = umapper0.fit_transform(iris.data)

Generating an initial embedding...
optimize_layout_euclidean_masked
head,tail shapes (150, 2) (150, 2)
fns_idx_pt []


In [2]:
print("Pinning embeddings of pts 13 and 14 to [-5,0] and [5,0]")
# pin embeddings of two data (13 and 14) to left and right of origin
pin_mask = np.ones_like(emb0)
pin_mask[13] = 0.0
pin_mask[14] = 0.0
emb0[13] = [-5.0, 0]
emb0[14] = [+5.0, 0]
print("Specify 'init' embedding for umapper2")
umapper1 = umap.UMAP(
    n_neighbors=50, learning_rate=0.5, random_state=12346, min_dist=0.001,
    init=emb0, n_epochs=2,
)
print("Embed with pin_mask[13] and pin_mask[14] zero-vectors")
emb1 = umapper1.fit_transform(iris.data, data_constrain=pin_mask)
print("emb0[11:15]\n",emb0[11:15])
print("emb1[11:15]\n",emb1[11:15])
print("\nGoodbye")

Pinning embeddings of pts 13 and 14 to [-5,0] and [5,0]
Specify 'init' embedding for umapper2
Embed with pin_mask[13] and pin_mask[14] zero-vectors
X.shape (150, 4)
data_constrain.shape (150, 2)
optimize_layout_euclidean_masked
head,tail shapes (150, 2) (150, 2)
pin_mask
sample 13 pin head[ 0 ] begins at -5.0
sample 13 pin head[ 1 ] begins at 0.0
sample 14 pin head[ 0 ] begins at 5.0
sample 14 pin head[ 1 ] begins at 0.0
fns_idx_pt [CPUDispatcher(<function optimize_layout_euclidean.<locals>.pin_mask_constraint at 0x7fa9e165ec10>)]
emb0[11:15]
 [[ 4.67359    3.3806326]
 [ 4.3936124  7.5779386]
 [-5.         0.       ]
 [ 5.         0.       ]]
emb1[11:15]
 [[ 4.804115   3.4301755]
 [ 4.2600307  7.352643 ]
 [-5.         0.       ]
 [ 5.         0.       ]]

Goodbye


In [3]:
print("Pinning embeddings of pts 13 and 14 to [-2,0] and [2,0]")
# pin embeddings of two data (13 and 14) to left and right of origin
# via a custom constraint.  inf get no-op, other values get fixed
import umap.constraints2 as con
import numba
infs0 = np.full_like(emb0, np.float32(np.inf), dtype=np.float32)
infs0[13,:] = [-2.0,0]
infs0[14,:] = [+2.0,0]
@numba.njit()
def constraint_idx_pt0(idx,pt):
    con.freeinf_pt(idx,pt, infs0)
    
constraints = {
    'idx_pt': constraint_idx_pt0,
}
# optional: set up the values to agree
#emb0[13] = [-2.0, 0]
#emb0[14] = [+2.0, 0]
# Here is the "move all points" version of con.freeinf
con.freeinf_pts(emb0, infs0)

assert np.all(emb0[13] == [-2.0,0])
assert np.all(emb0[14] == [+2.0,0])
print("Specify 'init' embedding for umapper2")
umapper2 = umap.UMAP(
    n_neighbors=50, learning_rate=0.5, random_state=12346, min_dist=0.001,
    init=emb0, n_epochs=2,
)
print("Embed with pin_mask[13] and pin_mask[14] zero-vectors")
emb2 = umapper2.fit_transform(iris.data, data_constrain=constraints)
print("emb0[11:15]\n",emb0[11:15])
print("emb2[11:15]\n",emb2[11:15])
print("\nGoodbye")

Pinning embeddings of pts 13 and 14 to [-2,0] and [2,0]
Specify 'init' embedding for umapper2
Embed with pin_mask[13] and pin_mask[14] zero-vectors
X.shape (150, 4)
data_constrain keys dict_keys(['idx_pt'])
optimize_layout_euclidean_masked
head,tail shapes (150, 2) (150, 2)
kk,k 0 idx_pt
fns_idx_pt [CPUDispatcher(<function constraint_idx_pt0 at 0x7fa9dc5a91f0>)]
emb0[11:15]
 [[ 4.67359    3.3806326]
 [ 4.3936124  7.5779386]
 [-2.         0.       ]
 [ 2.         0.       ]]
emb2[11:15]
 [[ 4.804111   3.430179 ]
 [ 4.2598176  7.352637 ]
 [-2.         0.       ]
 [ 2.         0.       ]]

Goodbye


In [4]:
print("grad constraint 13 and 14 on line y=x")
# this one has little help from umap.constraints2.py,
# so define the numba constraint functions here:
@numba.njit()
def y_eq_x_pt(idx, pt):
    avg = np.sum(pt) / pt.shape[0]
    pt.fill(avg)
@numba.njit()
def y_eq_x_grad(idx, pt, grad):
    # if we cannot assume pt satisfies constraints:
    #y_eq_x_pt(idx, pt)  # put pt onto 45-degree line
    # now tangent plane projection
    y_eq_x_pt(idx, grad) # gradient also lies on the 45-degree line

# pin embeddings of two data (13 and 14) to all-coords-equal line
constraints = {
    'idx_grad': y_eq_x_grad,
}
# init 13 and 14 to 45-degree line
emb0[13,:] = [-1.0, -1.0]
emb0[14,:] = [+1.0, +1.0]
umapper3 = umap.UMAP(
    n_neighbors=50, learning_rate=0.5, random_state=12346, min_dist=0.001,
    init=emb0, n_epochs=2,
)
print("Embed with pin_mask[13] and pin_mask[14] zero-vectors")
emb3 = umapper3.fit_transform(iris.data, data_constrain=constraints)
print("emb0[11:15]\n",emb0[11:15])
print("emb3[11:15]\n",emb3[11:15])
np.testing.assert_allclose(emb3[13,0], emb3[13,1])
np.testing.assert_allclose(emb3[14,0], emb3[14,1])
print("\nGoodbye")

grad constraint 13 and 14 on line y=x
Embed with pin_mask[13] and pin_mask[14] zero-vectors
X.shape (150, 4)
data_constrain keys dict_keys(['idx_grad'])
optimize_layout_euclidean_masked
head,tail shapes (150, 2) (150, 2)
kk,k 0 idx_grad
fns_idx_pt []
emb0[11:15]
 [[ 4.67359    3.3806326]
 [ 4.3936124  7.5779386]
 [-1.        -1.       ]
 [ 1.         1.       ]]
emb3[11:15]
 [[ 4.7785788  3.4856207]
 [ 4.2159853  7.4003115]
 [-0.91349   -0.91349  ]
 [ 1.1121976  1.1121976]]

Goodbye


In [14]:
import umap.constraints2 as con
import numba
print("spring force constraint 13 and 14 pulled towards (0,3), (0,-3)")
# This shows a "soft" constraint, with no point-projection step,
# and instead of projecting onto tangent space,
# the gradients get modified by a simple user force.

pin_idx = np.array([13,14], dtype=np.int32)
springs = np.array([0.1, 0.01], dtype=np.float32)   # note np==inf **would** have projection constraint
pin_pos = np.array([[0,3], [0,-3]], dtype=np.float32)
print("pin_idx (anchors)         ", numba.typeof(pin_idx), "\n", pin_idx)
print("springs (force constants) ", numba.typeof(springs),  "\n", springs)
print("pin_pos (anchor positions)", numba.typeof(pin_pos), "\n", pin_pos)

@numba.njit
def my_springs(idx, pt, grad):
    # pt is unconstrained
    con.springindexed_grad(idx,pt, grad, pin_idx, pin_pos, springs)

# second constraint (every pt inside simple box)
my_los = np.full(2, -5.0, dtype=np.float32) # x and y low bound <- -5.0
my_his = np.full(2, +5.0, dtype=np.float32)
@numba.njit
def my_box(idx, pt):  # 'idx_pt' argument list
    con.dimlohi_pt(pt, my_los, my_his)
# pin embeddings of two data (13 and 14) to all-coords-equal line
constraints = {
    'idx_pt':   my_box,
    'idx_grad': my_springs,
}
# init 13 and 14 "anywhere"
emb0[13,:] = [0, +5]
emb0[14,:] = [0, -5]
umapper4 = umap.UMAP(
    n_neighbors=50, learning_rate=0.5, random_state=12347, min_dist=0.001,
    init=emb0, n_epochs=4,
)
print("Embed with pin_mask[13] and pin_mask[14] zero-vectors")
emb4 = umapper4.fit_transform(iris.data, data_constrain=constraints)
print("emb0[11:15]\n",emb0[11:15])
print("emb4[11:15]\n",emb4[11:15])
print("pin_pos[0]", pin_pos[0], "distance:",np.linalg.norm(emb4[13] - pin_pos[0]))
print("pin_pos[1]", pin_pos[1], "distance:",np.linalg.norm(emb4[14] - pin_pos[1]))
print("\nGoodbye")

spring force constraint 13 and 14 pulled towards (0,3), (0,-3)
pin_idx (anchors)          array(int32, 1d, C) 
 [13 14]
springs (force constants)  array(float32, 1d, C) 
 [0.1  0.01]
pin_pos (anchor positions) array(float32, 2d, C) 
 [[ 0.  3.]
 [ 0. -3.]]
Embed with pin_mask[13] and pin_mask[14] zero-vectors
X.shape (150, 4)
data_constrain keys dict_keys(['idx_pt', 'idx_grad'])
optimize_layout_euclidean_masked
head,tail shapes (150, 2) (150, 2)
kk,k 0 idx_pt
kk,k 1 idx_grad
fns_idx_pt [CPUDispatcher(<function my_box at 0x7fa9cf813940>)]
emb0[11:15]
 [[ 4.67359    3.3806326]
 [ 4.3936124  7.5779386]
 [ 0.         5.       ]
 [ 0.        -5.       ]]
emb4[11:15]
 [[ 3.1389043   0.33609653]
 [ 3.0078042   5.        ]
 [-0.2341685   3.6357818 ]
 [ 0.46267805 -3.4742126 ]]
pin_pos[0] [0. 3.] distance: 0.67753476
pin_pos[1] [ 0. -3.] distance: 0.662532

Goodbye


In [6]:
import numpy as np
a = tuple()
print(*a)
print(type(a),a, a.count(1), len(a))  # a.index(1) ValueError
a = tuple((np.sort,1,2,3))
print(type(a),a, a.count(1), a.index(1))
print(a[0], a[1], a[1:], type(a[1:]))
a = tuple((np.sort,1))
print(a[0], a[1], a[1:], type(a[1:]))
a = (3.14,)
print(a[0], a[1:], type(a[1:]), len(a), len(a[1:]))
print(*a)


<class 'tuple'> () 0 0
<class 'tuple'> (<function sort at 0x7faa1425bb80>, 1, 2, 3) 1 1
<function sort at 0x7faa1425bb80> 1 (1, 2, 3) <class 'tuple'>
<function sort at 0x7faa1425bb80> 1 (1,) <class 'tuple'>
3.14 () <class 'tuple'> 1 0
3.14


In [7]:
a = np.array([[1,2],[3,4]])
b = np.array([[0,0],[1,1]])
def foo(a,b):
    a = np.where( b==0, a, 13)  # a is NOT modified-in-place
    return a
def bar(a,b):
    a[:,:] = np.where( b==0, a, 15)  # a IS modified-in-place
    #^^^^^
    return a
print(b==0)
#
# foo DOES NOT modify in place
#
aa = a.copy()
x = foo(aa,b)
#print("a\n", a, "\naa\n", aa, "\nx\n", x)
assert np.all(aa==a)
assert np.all(x[1] == 13)
#
# but bar DOES
#
aa = a.copy()
x = bar(aa,b)
#print("a\n", a, "\naa\n", aa, "\nx\n", x)
assert np.all(aa==x)
assert np.all(x[1] == 15)


[[ True  True]
 [False False]]


In [8]:
a = (1,2,3)
b = (4,5)
a += b
a += (6,)
#a.ext((7,))
print(type(a),a)
a = [1,2,3]
a.extend([4,5])
a.extend([7])
a.extend([])
a+=[8,9]
print(type(a),a)
print(tuple(tuple(a)))
#

<class 'tuple'> (1, 2, 3, 4, 5, 6)
<class 'list'> [1, 2, 3, 4, 5, 7, 8, 9]
(1, 2, 3, 4, 5, 7, 8, 9)


In [9]:
import numba
@numba.njit()
def _fn_idx_pt_noop(idx,pt):
    print("no-op")
    return None
def _chain_idx_pt(fs, inner=None):
    # for f in tuple 'fs' invoke f, where fs is a tuple of jitted functions(idx,pt)
    if len(fs) == 0:
        assert inner is None
        return _fn_idx_pt_noop
    head = fs[0]
    #@numba.njit()
    #def wrap(idx,pt):
    #    #return head(inner(idx,pt))
    #    if inner is not None:
    #       inner(idx,pt)
    #    head(idx,pt)
    if inner is None:
        @numba.njit()
        def wrap(idx,pt):
            head(idx,pt)
    else:
        @numba.njit()
        def wrap(idx,pt):
            inner(idx,pt)
            head(idx,pt)
    if len(fs) > 1:
        tail = fs[1:]
        return _chain_idx_pt(tail, wrap)
    else:
        return wrap
@numba.njit()
def f1(idx,pt):
    print("add",idx)
    pt += idx
@numba.njit()
def f2(idx,pt):
    print("mul",idx)
    pt *= idx

g = _chain_idx_pt(tuple())
idx = np.array(2); pt=np.array(1); print(idx,pt); 
g(idx,pt); print(idx,pt)

g = _chain_idx_pt((f1,))
idx = np.array(2); pt=np.array(1); print(idx,pt); 
g(idx,pt); print(idx,pt)

g = _chain_idx_pt((f1,f2))
idx = np.array(2); pt=np.array(1); print(idx,pt); 
g(idx,pt); print(idx,pt)

# does it work with [...]? yes
g = _chain_idx_pt([f1,f2,f1,f1])
idx = np.array(2); pt=np.array(1); print(idx,pt); 
g(idx,pt); print(idx,pt)
print(numba.typeof(g))

@numba.njit()
def gpass(f,idx,pt):
    print("gpass",f(idx,pt))
idx = np.array(2); pt=np.array(1); print(idx,pt); 
gpass(g,idx,pt)
print(idx,pt)

2 1
no-op
2 1
2 1
add 2
2 3
2 1
add 2
mul 2
2 6
2 1
add 2
mul 2
add 2
add 2
2 10
type(CPUDispatcher(<function _chain_idx_pt.<locals>.wrap at 0x7fa9dc547550>))
2 1
add 2
mul 2
add 2
add 2
gpass None
2 10


In [10]:
a = {1:2, 2:4, 1:3}
a


{1: 3, 2: 4}

In [11]:
a = np.ones((3,3))
x = np.full_like(a, np.inf, dtype=np.float32)
print(x)
print(type(x), str(x))

[[inf inf inf]
 [inf inf inf]
 [inf inf inf]]
<class 'numpy.ndarray'> [[inf inf inf]
 [inf inf inf]
 [inf inf inf]]


In [12]:
#a = [[1,2],[3],[4,5,6]]
#a.flatten()  # no flatten function